In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import os
import numpy as np
import glob

In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
inpath = r"T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\Trips"

In [6]:
year = 2023

In [25]:
month = 7

In [10]:
def monthend(m=month, y=year):
    if m in [1, 3, 5, 7, 8, 10, 12]:
        enddate = 31
    elif m in [4, 6, 9, 11]:
        enddate = 30
    elif (m == 2) and ((y%4 == 0 and y%100 != 0) or (y%400 == 0)):
        enddate = 29
    else:
        enddate = 28
    return enddate        

In [26]:
enddate = monthend(month)

In [27]:
file = "trips_peace_health_rides_0"+str(month) +"_01_"+str(year)+"-0"+str(month)+"_"+str(enddate)+"_"+str(year)+".csv"

In [28]:
file

'trips_peace_health_rides_05_01_2022-05_31_2022.csv'

In [29]:
odf = pd.read_csv(os.path.join(inpath, file))
odf.head()

,User ID,Route ID,Bike Region ID,Payment Plan,Start Hub,Start Area,Start Special Area,Start Latitude,Start Longitude,Start Date,...,Bike ID,Bike Name,Distance [Miles],Duration,Rental Access Path,Multiple Rental,Ride cost,Fees,Bonuses,Total cost
0,2231320,14308713,NaN,UO Monthly Plan,NaN,NaN,NaN,44.048988333333334,-123.08251166666666,2022-05-01,...,19850,0096,1.01,00:07:45,NaN,False,0.0,0.0,0.0,0.0
1,2222901,14308715,NaN,UO Monthly Plan,PeaceHealth University District,Phase 1 System Area,NaN,44.047641666666664,-123.08187833333334,2022-05-01,...,19747,0217,1.45,00:09:56,NaN,False,0.0,0.0,0.0,0.0
2,2038600,14308717,NaN,UO Monthly Plan,19th and Agate,Phase 1 System Area,NaN,44.03908,-123.06958666666667,2022-05-01,...,19722,0192,0.86,00:06:34,NaN,False,0.0,0.0,0.0,0.0
3,2223054,14308720,NaN,UO Single Ride,8th & Ferry,Phase 1 System Area,NaN,44.05105833333333,-123.08499833333333,2022-05-01,...,19799,0269,1.34,00:08:50,NaN,False,0.0,0.0,0.0,0.0
4,2207367,14308722,NaN,UO Monthly Plan,PeaceHealth University District,Phase 1 System Area,NaN,44.04763,-123.08197,2022-05-01,...,19654,0124,0.78,00:05:39,NaN,False,0.0,0.0,0.0,0.0


In [30]:
df = odf 

In [31]:
df = df[(df['Start Latitude'] != " - ") | (df['End Latitude'] != " - " ) | (df['Start Longitude'] != " - ") | (df['End Longitude'] != " - ")]

In [21]:
def getLoc(x):
    if x is not None:
        loc = x.raw['address']
        a = [key for key, value in loc.items()][0]
        b = [key for key, value in loc.items()][1]
        if  a == 'house_number':
            res = loc['road'] + ' ' + loc['house_number']
        else:
            res = loc[a]
            if res == "University of Oregon":
                res = "UO " + loc[b]
    else:
        res = ""
    return res

In [19]:
def review_hubs(re_df):
    for hub in ["Start", "End"]:
        if hub+' Hub' in re_df.columns:
            if re_df[re_df[hub+' Hub'].isnull()].shape[0] > 0:
                re = re_df[re_df[hub+' Hub'].isnull()][[hub+' Latitude', hub+' Longitude']]
            else:
                re = re_df[re_df[hub+' Hub'].str.isnumeric()][[hub+' Latitude', hub+' Longitude']]
        else:
            re = re_df[re_df[hub+'.Hub'].str.isnumeric()][[hub+'.Latitude', hub+'.Longitude']]
        
        if re.shape[0] > 0:
            re.columns = ['Lat', 'Lon']
            re["geom"] = re["Lat"].map(str) + ',' + re['Lon'].map(str)
            locator = Nominatim(user_agent="measurements", timeout=10)
            rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)
            tqdm_notebook.pandas()
            re['address'] = re['geom'].progress_apply(rgeocode)
            re['name'] =re.address.apply(getLoc)
            if hub+' Hub' in re_df.columns:
                if re_df[re_df[hub+' Hub'].isnull()].shape[0] > 0:
                    re_df.loc[re_df[hub+' Hub'].isnull(), hub+' Hub'] = re.name.values
                else:
                    re_df.loc[re_df[hub+' Hub'].str.isnumeric(), hub+' Hub'] = re.name.values  
            else:
                re_df.loc[re_df[hub+'.Hub'].str.isnumeric(), hub+'.Hub'] = re.name.values
    return re_df

In [32]:
re_df = review_hubs(df)

  0%|          | 0/4063 [00:00<?, ?it/s]

C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


  0%|          | 0/4616 [00:00<?, ?it/s]

C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [33]:
outpath = r"T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\Output\review"

In [34]:
re_df.to_csv(os.path.join(outpath, file), index=False)